In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.streetcheck.co.uk/postcode/startingwith/e17'

In [3]:
r = requests.get(url)

In [5]:
r.status_code
# returns 403 error which means The HTTP 403 Forbidden client error status response code indicates that the server understood the request but refuses to authorize it. 

403

In [6]:
import os

In [ ]:
chrome_driver_path

In [9]:
os.path.expanduser('~')

'/Users/makaibe'

In [ ]:
/Users/makaibe/Dropbox/My Mac (Nwamakas-MacBook-Pro.local)/Documents/chromedriver